<p align='center'>

![alt text](Modak_LogoHorizontal_AmarilloVerde_1_(1).jpg)

<p>

<h1 align='center'>
 <b> DATA CHALLENGE</b>
</h1>


<h2 align='center'>
 Report  - Data Engineer - MODAK
</h1>

---------------------------------------------------------------------------

Welcome to the ETL and EDA of this project. In this file I will show you step by step the examination, normalization, short analysis and cleaning of three datasets:

1. allowance_events (JSON file)
2. payment_schedule_backend_table (CSV file)
3. allowance_backend_table (CSV file)

In this project I have a Data Engineer rol, with the porpouse to help the backend team to solve some issues in there data tables, you can find the Project instructions and goal in this repository READ.MD.

## <CENTER> 1. allowance_events 
#### <CENTER> -- JSON file --

## Data Dictionary

### 1. allowance_events (JSON)
This dataset captures the creation and updates of user allowances during the affected period.

○ Fields:
- **event.name**: The type of event, either allowance.created or
allowance.edited.
- **event.timestamp**: Timestamp of the event.
- **user.id**: Unique identifier for the user.
- **allowance.amount**: Allowance amount.
- **allowance.scheduled.frequency**:Frequency of the allowance
daily, weekly, biweekly, or monthly.
- **allowance.scheduled.day**:
    - "daily" for daily frequency.
    - Day of the week for weekly/biweekly schedules.
    - "1st" or "15th" for monthly schedules.

### Importing and Normalizing the Data

In [1]:
import pandas as pd
import json
import datetime
from datetime import datetime, date, timedelta
import pendulum
from dateutil.relativedelta import relativedelta, FR, MO ,TU ,WE ,TH ,SA ,SU

In the following code, the JSON file is imported and converted to a pandas DataFrame, in order to explored and analyzed.

In [2]:
#Load the JSON data
with open('allowance_events.json','r') as f:
    data = json.load(f)
#Create a DataFrame from the JSON data
allowance_events = pd.DataFrame(data)
#organize and distribute the data frame in proper columns
allowance_events['user_ID'] = [list(line.values())[0] for line in allowance_events['user']]
allowance_events['TIMESTAMP'] = [list(line.values())[0] for line in allowance_events['event']]
allowance_events['allowance_TYPE']=[list(line.values())[1] for line in allowance_events['event']]
allowance_events['allowance_TYPE']=[list(line.split('.'))[1] for line in allowance_events['allowance_TYPE']]
allowance_events['FREQUENCY']=[list(list(line.values())[0].values())[0] for line in allowance_events['allowance']]
allowance_events['DAY']=[list(list(line.values())[0].values())[1] for line in allowance_events['allowance']]
allowance_events['AMOUNT']=[list(line.values())[1] for line in allowance_events['allowance']]
#The columns who previously hold the data are deleted since now the data is distributed and organized in new columns
allowance_events.drop(columns=['user', 'event', 'allowance'], inplace=True)

In [3]:
#The data per column is converted to its respective types: DateTime - Category
allowance_events['TIMESTAMP']= pd.to_datetime(allowance_events['TIMESTAMP'], format="%Y-%m-%d %H:%M:%S")
allowance_events['user_ID']= allowance_events['user_ID'].astype('category')
allowance_events['allowance_TYPE']= allowance_events['allowance_TYPE'].astype('category')
allowance_events['FREQUENCY']= allowance_events['FREQUENCY'].astype('category')
allowance_events['DAY']= allowance_events['DAY'].astype('category')
#Since the AMOUNT column has alredy a int64 type there is no need to chage it.

Here we can see the general information about this dataset:
- It has 6208 rows and 6 columns.
- There are no null values in any column.
- The *user_id, allowance_TYPE, FREQUENCY*, and *DAY* columns have categorical values, *TIMESTAMP* date & time values and the *AMOUNT* column has integer values.

In [4]:
allowance_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6208 entries, 0 to 6207
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_ID         6208 non-null   category      
 1   TIMESTAMP       6208 non-null   datetime64[ns]
 2   allowance_TYPE  6208 non-null   category      
 3   FREQUENCY       6208 non-null   category      
 4   DAY             6208 non-null   category      
 5   AMOUNT          6208 non-null   int64         
dtypes: category(4), datetime64[ns](1), int64(1)
memory usage: 215.2 KB


Here we can see a small view of the data that every column contains:

In [5]:
allowance_events.tail(6) #code to see the last 5 rows.

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT
6202,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 06:59:24,edited,biweekly,thursday,5
6203,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 06:59:25,edited,biweekly,thursday,5
6204,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:00:05,edited,daily,daily,5
6205,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:00:05,edited,daily,daily,5
6206,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:01:13,edited,biweekly,saturday,5
6207,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:01:14,edited,biweekly,saturday,5


--------

### Duplicated and Unique values:

The following codes shows the unique values quantity in each column of the dataset: 

In [6]:
allowance_events.nunique()

user_ID           2879
TIMESTAMP         5191
allowance_TYPE       2
FREQUENCY            4
DAY                 10
AMOUNT              61
dtype: int64

The total amount of rows in this dataset is 6208, as we could see previously on the dataset info. However, the numbre of rows doesn't coinceed with the unique values in the user_ID column(2879). This is expected, because this table is supose to storage all the times the users edited their allowance settings, since the creation to the last edition.

The unique user_id quantity matches the amount of rows with "created" as the *allowance_TYPE* value, what makes sense since a new user id should only be created if the allowance was generated for the first time. 


In [7]:
len(allowance_events.user_ID.unique())

2879

In [8]:
print(f'Rows with allowance_TYPE "created": {len(allowance_events[allowance_events["allowance_TYPE"]== "created"])}')
print(f'Rows with allowance_TYPE "edited": {len(allowance_events[allowance_events.allowance_TYPE== "edited"])}')

Rows with allowance_TYPE "created": 2879
Rows with allowance_TYPE "edited": 3329


Here we can observed the period of time that this dataset covers:

In [9]:
lisTIMESTAMP=allowance_events.TIMESTAMP.unique()
print(f'First date:{lisTIMESTAMP.min()}\nLast date:{lisTIMESTAMP.max()}')

First date:2024-07-25 11:18:14
Last date:2024-12-03 05:07:41


The *allowance_TYPE* column specifies if the settings were cerated for the first time or edited:

In [10]:
allowance_events.allowance_TYPE.unique()

['created', 'edited']
Categories (2, object): ['created', 'edited']

There are 4 types of frequencies a user can choose in the allowance settings:

- **Weekly**: Once a week.
- **biweekly**: Once every 15 days.
- **monthly**: Once a month.
- **daily**: Every day

In [11]:
allowance_events.FREQUENCY.unique()

['weekly', 'biweekly', 'monthly', 'daily']
Categories (4, object): ['biweekly', 'daily', 'monthly', 'weekly']

There are 10 options for day a user can choose in the allowance settings:

- **Any day of the week** : monday, tuesday,  wednesday, thursday, friday, saturday, sunday (for weekly or biweekly frequency).
- **first_day**: always the 1th (only for month frequency).
- **fifteenth_day:** always the 15th (only for month frequency).
- **daily**: Every day

In [12]:
allowance_events.DAY.unique()

['sunday', 'friday', 'thursday', 'saturday', 'first_day', 'fifteenth_day', 'monday', 'daily', 'wednesday', 'tuesday']
Categories (10, object): ['daily', 'fifteenth_day', 'first_day', 'friday', ..., 'sunday', 'thursday', 'tuesday', 'wednesday']

Thanks to the following code, we can see that the amount of money per allowance can be between 0 and 200:

In [13]:
lis=allowance_events.AMOUNT.unique()
lis.sort()
lis

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  18,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  32,  35,  36,  40,  45,  48,  50,  52,  55,  57,
        60,  70,  75,  80,  83,  84,  94,  95, 100, 105, 114, 115, 120,
       125, 130, 147, 150, 155, 160, 180, 190, 200], dtype=int64)

------

### Adding a Next_payment_day Column:

The following code is a large function created to determinate the next payment day per each allowance, base on the frequency and day selected by the user. The porpose of this function creation is to check the *next_payment_day* column from the allowance_backend_table, which seems to have incorrect values.

*Explanation of **normalize_next_payment_day**:*

The result sought by this function is the next day on which the payment will be made, based on the configuration chosen by the user. Normalize_next_payment_day receives two parameters, f(frequency) and d (day). If the frequency indicates 'weekly', the code would look for the next week day  specified in the d parameter, while if the frequency is 'biweekly', the code would seek for the date's day in 15 days. Meanwhile if the frequency is determinate as 'monthly', the d paramenter will definde if the result is 1 or 15, since those are the only options allowed. And finally, if the frequency or is 'daily', the result would be the next day. The function is base on the *TIMESTAMP* column values, since its the last real date when the allowance configuration was changed.

It is important to clarify that the function consideres the date of today as Dicember the 3rd, 2024; as it has been specify in the challenge instructions.

Here the function is apply to the dataset:

In [14]:
(pendulum.instance((allowance_events['TIMESTAMP'][2109]).date() + relativedelta(weekday=MO(-1))).next(pendulum.MONDAY) + timedelta(days=7))

Date(2024, 11, 4)

In [15]:
def normalize_next_paymente_day(f,d,e):

    today=datetime(2024,12,3).date()

    if d == 'monday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.MONDAY)).day           #This line detect the following MONDAY after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=MO(-1))).next(pendulum.MONDAY) + timedelta(days=7))           #This code asume the last MONDAY was the last allowance schedueled and looks for next MONDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=MO(-1))).next(pendulum.MONDAY) + timedelta(days=7)).day
    
    elif d == 'tuesday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.TUESDAY)).day           #This line detect the following TUESDAY after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=TU(-1))).next(pendulum.TUESDAY) + timedelta(days=7))           #This code asume the last TUESDAY was the last allowance schedueled and looks for next TUESDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=TU(-1))).next(pendulum.TUESDAY) + timedelta(days=7)).day
    
    elif d == 'wednesday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.WEDNESDAY)).day           #This line detect the following WEDNESDAY after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=WE(-1))).next(pendulum.WEDNESDAY) + timedelta(days=7))           #This code asume the last WEDNESDAY was the last allowance schedueled and looks for next WEDNESDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=WE(-1))).next(pendulum.WEDNESDAY) + timedelta(days=7)).day
    
    if d == 'thursday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.THURSDAY)).day           #This line detect the following THURSDAY after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=TH(-1))).next(pendulum.THURSDAY) + timedelta(days=7))           #This code asume the last THURSDAY was the last allowance schedueled and looks for next THURSDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=TH(-1))).next(pendulum.THURSDAY) + timedelta(days=7)).day

    if d == 'friday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.FRIDAY)).day           #This line detect the following Friday after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=FR(-1))).next(pendulum.FRIDAY) + timedelta(days=7))           #This code asume the last FRIDAY was the last allowance schedueled and looks for next FRIDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=FR(-1))).next(pendulum.FRIDAY) + timedelta(days=7)).day
    
    elif d == 'saturday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.SATURDAY)).day           #This line detect the following SATURDAY after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=SA(-1))).next(pendulum.SATURDAY) + timedelta(days=7))           #This code asume the last SATURDAY was the last allowance schedueled and looks for next SATURDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=SA(-1))).next(pendulum.SATURDAY) + timedelta(days=7)).day
    
    elif d == 'sunday':
        if f == 'weekly':
            rta= (pendulum.instance(today).next(pendulum.SUNDAY)).day           #This line detect the following SUNDAY after the actual day:2024/12/3
        else:
            rta1= (pendulum.instance((allowance_events['TIMESTAMP'][e]).date() + relativedelta(weekday=SU(-1))).next(pendulum.SUNDAY) + timedelta(days=7))           #This code asume the last SUNDAY was the last allowance schedueled and looks for next SUNDAY in 15 days.
            rta=rta1.day
            if rta1<today:
                rta=(pendulum.instance(today + relativedelta(weekday=SU(-1))).next(pendulum.SUNDAY) + timedelta(days=7)).day
    
    
    elif f == 'daily':
        rta= (pendulum.instance(today) + timedelta(days=1)).day         #This code just sum 1 day to the actual date, because its daily frequency.
    
    elif f == 'monthly': 
        if d == 'fifteenth_day':
            rta=15          #This code just complete with 15, since the option specifies the allowance to be scheduled on the 15th every month.
        else:           #If none of the options avoid were chosen, it means the it says "The 1th"
            rta=1           #This code just complete with 1, since the option specifies the allowance to be scheduled on the 1th every month.

    return rta

In [16]:
allowance_events['next_payment_day']= [normalize_next_paymente_day(allowance_events['FREQUENCY'][e],allowance_events['DAY'][e],e) for e in range(0,len(allowance_events))]

In [17]:
allowance_events.tail(10)

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day
6198,89e8521e-a9ba-4d5c-9c47-602dc0ec5218,2024-11-23 13:16:49,edited,biweekly,wednesday,5,4
6199,89e8521e-a9ba-4d5c-9c47-602dc0ec5218,2024-11-23 13:16:50,edited,biweekly,wednesday,5,4
6200,89e8521e-a9ba-4d5c-9c47-602dc0ec5218,2024-12-01 19:02:53,edited,biweekly,wednesday,10,11
6201,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 06:59:07,created,biweekly,monday,5,16
6202,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 06:59:24,edited,biweekly,thursday,5,12
6203,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 06:59:25,edited,biweekly,thursday,5,12
6204,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:00:05,edited,daily,daily,5,4
6205,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:00:05,edited,daily,daily,5,4
6206,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:01:13,edited,biweekly,saturday,5,14
6207,a0cd326b-bca3-454b-9ca1-127fe82e4eba,2024-11-08 07:01:14,edited,biweekly,saturday,5,14


---------

### Filtering allowance_events

After analicing and cleaning the two tables from the Backend team, I'm going to mearge the dataset to evaluate the discrepancies. To achived this, I'm going to filter the **allowance_events** dataset in order to only keep the most recent rows per *user_id*. So I will end up with only unique *user_id* values and there last edition in there allowance settings.

After analyzing and cleaning the two tables from the Backend team, I will merge the dataset to evaluate any discrepancies. To achieve this, I will filter the **allowance_events** table to keep only the most recent rows by *user_id*. So I'll end up with just unique *user_id* values ​​and their last edit in their allowance configuration.

*Explanation of the code below:*

1. First the table is sorted by the column *TIEMSTAMP* in a descendig way, so the first row is the last updated, the one with the newest date and time.
2. Then there is a small view of the 3329 rows with repeted *user_id*, whichones I have the intention to keep only the most updated ones.

    There is a clear example between the row *4014* and *4017* of a *user_id* who has 4 rows, one for each of their allowance updated settings.
3. The following code consisted in deleting the duplicated *user_id* rows, by keeping only the first one that appears. Since the dataset is sorted, the code keeps the updatest row.
4. The last code line proves that there is no longer duplicated *user_id* values in the table.


In [18]:
allowance_events.sort_values(by=['TIMESTAMP'],ascending=False, inplace=True, ignore_index=True)

In [19]:
allowance_events[allowance_events.duplicated(subset=['user_ID'])].sort_values(by=['user_ID'])

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day
2415,00139bdc-92b3-4ebd-af94-285acf2fd376,2024-10-29 13:19:28,created,weekly,tuesday,20,10
4014,0035bbe5-5034-40b6-aa8f-b50047f09dea,2024-09-28 06:49:22,edited,weekly,saturday,5,7
4015,0035bbe5-5034-40b6-aa8f-b50047f09dea,2024-09-28 06:43:53,edited,weekly,sunday,5,8
4016,0035bbe5-5034-40b6-aa8f-b50047f09dea,2024-09-28 06:43:53,edited,weekly,sunday,5,8
4017,0035bbe5-5034-40b6-aa8f-b50047f09dea,2024-09-28 06:43:25,created,weekly,monday,5,9
...,...,...,...,...,...,...,...
2497,fec83afc-5a60-480b-a52e-28101c712372,2024-10-28 03:51:49,created,weekly,friday,20,6
4446,ffa45e48-7b51-4953-a853-564e7ccdce3b,2024-09-21 14:33:22,edited,weekly,wednesday,5,4
4450,ffa45e48-7b51-4953-a853-564e7ccdce3b,2024-09-21 14:02:40,created,biweekly,wednesday,5,11
2469,ffbee036-6d3c-4072-8998-3f0988353077,2024-10-28 13:57:40,created,biweekly,friday,20,13


In [20]:
allowance_events.drop_duplicates(subset=['user_ID'], keep='first', inplace=True, ignore_index=True)

In [21]:
allowance_events[allowance_events['user_ID'].duplicated()] #Just to check, there are no more user_ID duplicated

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day


In [22]:
allowance_events

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day
0,a4fffebe-5b35-4ded-b83b-14852492ddca,2024-12-03 05:07:41,edited,weekly,monday,5,9
1,31f302d1-ee38-4cdc-8943-845f32e23d13,2024-12-03 04:57:13,edited,weekly,tuesday,7,10
2,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,2024-12-03 04:28:10,created,monthly,fifteenth_day,20,15
3,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,2024-12-02 22:39:37,created,weekly,friday,11,6
4,397f8b21-3af1-4a08-81eb-e085eb1160ca,2024-12-02 22:36:34,created,weekly,friday,11,6
...,...,...,...,...,...,...,...
2874,c5a7c5f7-6c1e-4ea0-919a-f7efdf4e7cc9,2024-07-25 11:55:57,edited,weekly,friday,1,6
2875,febf7ed7-a2e0-40b8-899b-3fb511a91d32,2024-07-25 11:37:03,created,monthly,first_day,1,1
2876,038c6ae6-ff6c-4b68-b33d-847a3ca50601,2024-07-25 11:36:43,created,biweekly,friday,2,13
2877,78fea998-707b-4a93-b92c-d84362165097,2024-07-25 11:36:14,created,weekly,friday,1,6


------------

## <CENTER> 2. payment_schedule_backend_table
#### <CENTER> - CSV file -

## Data Dictionary

#### 2. payment_schedule_backend_table (CSV)
This table contains the payment schedule records. It has been observed that the
payment_date field may contain errors or inconsistencies.

○ Fields:
- **user_id**: Unique identifier for the user (corresponds to user.id inevents).
- **payment_date**: Scheduled payment day. This field is suspected tocontain discrepancies.

### Importing and Normalizing the Data

In [23]:
payment_schedule_backend_table=pd.read_csv('payment_schedule_backend_table.csv')

Here we can see the general information about this dataset:
- It has 2139 rows and 2 columns.
- There are no null values in any column.
- The *user_id* column has object values, that would be convert to categorical, and the *payment_date* column has integer values.

In [24]:
payment_schedule_backend_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2139 entries, 0 to 2138
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       2139 non-null   object
 1   payment_date  2139 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.5+ KB


In [25]:
payment_schedule_backend_table['user_id']= payment_schedule_backend_table['user_id'].astype('category')

Here we can see a small view of the data that both column contains:

In [26]:
payment_schedule_backend_table

,user_id,payment_date
0,061d8039-b332-40ca-b906-3514b06411c2,22
1,0ca92d01-feed-42e5-b90c-299b615f9d3f,22
2,160bcd2d-3ab0-4a60-9bfa-ccfcc1ee17e4,22
3,1649fee7-c424-4fb7-ac84-db36e15e042a,22
4,1d647703-874d-4744-b4de-6a5212518aa8,22
...,...,...
2134,f184a04d-f1ed-4f70-93a4-19426423a476,14
2135,f348b2f8-d9ad-40fe-9e1c-dd7f84f00396,14
2136,f9636ff4-a3d9-46f4-8418-bc8b512d6eee,14
2137,f970b14b-7aa2-43b3-b626-287578e521e6,14


--------

### Duplicated and Unique values:

The following codes shows the unique values quantity in each column of the dataset: 

In [27]:
print(f'Here we can see the amuount of unique values in each column:\n{payment_schedule_backend_table.nunique()}')
print(f'\nHere we see the amuount of rows in the dataset: {payment_schedule_backend_table.shape[0]}')
print(f'Diference between total dataset rows and user_id unique values: {payment_schedule_backend_table.shape[0] - len(payment_schedule_backend_table.user_id.unique())}')

Here we can see the amuount of unique values in each column:
user_id         2126
payment_date      31
dtype: int64

Here we see the amuount of rows in the dataset: 2139
Diference between total dataset rows and user_id unique values: 13


The *payment_date* column is expected to have only 31 unique values, since the column specifies the day of the month scheduled for payment. However, we still need to check if each value it is aligned with the *next_paid_day* column of Allowance_backend_table, which also needs to be checked first.

Meanwhile, we can see a clear difference between the number of unique *usier_id* values ​​and the total number of rows in this table. This proves that there is more than one row dedicated to a user, this needs to be fixed as there should only be one row with a single payment date per *user_id*.

##### Duplicated:
In the following output we can see that there are no duplicate rows in this table, only duplicate *user_id* values. To fix this problem, I am going to delete the rows where *user_id* are repeated and after extracting the actual *payment_date* values, According to the other two datasets, I will correct the *payment_date* column.

In [28]:
payment_schedule_backend_table[payment_schedule_backend_table.duplicated()]

,user_id,payment_date


In [29]:
print('These are the 13 duplicated rows that will be delete from the table:')
duplicate=payment_schedule_backend_table[payment_schedule_backend_table['user_id'].duplicated()].index
payment_schedule_backend_table[payment_schedule_backend_table['user_id'].duplicated()]

These are the 13 duplicated rows that will be delete from the table:


,user_id,payment_date
1274,d299d804-0d05-48c8-ad62-601d54438e34,27
1310,72e8fe3f-0dd9-44e3-8775-660bacb17310,5
1498,19d933a6-2810-4c08-b3d6-d68ef3c6eac3,7
1543,c091a91b-c6fd-44cb-8ac7-fe3fdbafccea,7
1592,72190913-1208-4333-aaf4-ea7d04dccacd,11
1598,9221dccf-44f3-446d-b3ce-19f9123ea627,11
1629,172fe923-347e-4d0e-ba82-1e9969aaadde,3
1639,abf5cf1e-e81a-4b5f-9d5b-649fa10d8f81,3
1652,42f61042-3d31-42ff-9f1a-508716069464,20
1785,d769685e-9046-4ffe-a51d-829ec7f1d877,25


In the following code, rows with duplicate *user_id* are removed:

In [30]:
payment_schedule_backend_table.drop(labels=duplicate, inplace=True)

In [31]:
payment_schedule_backend_table.shape

(2126, 2)

## <CENTER> 3. allowance_bakend_table 
#### <CENTER> -- CSV file --

## Data Dictionary
### 3. allowance_backend_table (CSV)
This table contains backend records of allowances during the affected period. It has been observed that the next_payment_day field in this table may contain errors or inconsistencies.

○ Fields:
- **uuid**: Unique identifier for the user (corresponds to *user.id* in events).
- **creation_date**: Timestamp when the record was created.
- **frequency**: Allowance frequency (daily, weekly, biweekly,
monthly).
- **day**: Scheduled day for the allowance.
- **updated_at**: Timestamp of the most recent update to the record.
- **next_payment_day**: The next scheduled payment day as a float (1 to
31). This field is suspected to contain discrepancies.
- **status**: indicates if the allowance is currently enabled or disabled.

### Importing and Normalizing the Data

In [32]:
allowance_backend_table=pd.read_csv('allowance_backend_table.csv')

Dataset Preview:

In [33]:
allowance_backend_table.tail(12)

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
2870,7e3bb803-f698-41b6-a126-b657f138c2dc,1726980961,monthly,first_day,1726980961,1,enabled
2871,c5f85d34-50f3-4d78-9259-abb7a9dc73cd,1730232745,weekly,friday,2024-11-28T01:12:00.574956849Z,29,disabled
2872,cca4875d-e050-42de-a22e-bc6d4c46c428,1728466309,weekly,sunday,2024-12-01T08:01:46.525069902Z,8,enabled
2873,8aa1bbb5-2e65-475c-a699-a81c9e51dfd3,1731943819,biweekly,friday,2024-11-26T08:45:19.8148501Z,6,enabled
2874,38ca8952-bd2d-4e33-806f-3b77f0fb02fe,1726591433,biweekly,monday,2024-10-08T22:38:14.787882594Z,14,disabled
2875,b3b99410-40c0-4574-b6cc-b0769c992971,1726382886,biweekly,wednesday,2024-10-25T08:00:46.017618264Z,6,enabled
2876,c6dde30f-f54d-400c-ad5c-d87728d81731,1730686297,biweekly,saturday,2024-11-23T20:33:08.809249933Z,7,enabled
2877,2e26d7e7-b65e-4468-8096-5bed0ef99988,1729098435,biweekly,monday,2024-11-28T08:00:30.184784321Z,9,enabled
2878,adec35e8-3699-45e5-b675-4a6504b296de,1722370541,weekly,friday,2024-08-16T00:48:04.406999984Z,16,disabled
2879,e8505cf5-856d-4518-9cec-ffe4b72d9a45,1722370555,weekly,friday,2024-08-16T00:48:10.788978083Z,16,disabled


In [34]:
allowance_backend_table['uuid']= allowance_backend_table['uuid'].astype('category')
allowance_backend_table['frequency']= allowance_backend_table['frequency'].astype('category')
allowance_backend_table['day']= allowance_backend_table['day'].astype('category')
allowance_backend_table['status']= allowance_backend_table['status'].astype('category')

Here we can see the general information about this dataset:
- It has 2882 rows and 7 columns.
- There are no null values in any column.
- The *uuid, allowance_TYPE, frequency, day, updated_at, status* columns have categorical values, then *creation_date*,*next_payment_day* column have integer values and *updated_at* as object value.

In [35]:
allowance_backend_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2882 entries, 0 to 2881
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   uuid              2882 non-null   category
 1   creation_date     2882 non-null   int64   
 2   frequency         2882 non-null   category
 3   day               2882 non-null   category
 4   updated_at        2882 non-null   object  
 5   next_payment_day  2882 non-null   int64   
 6   status            2882 non-null   category
dtypes: category(4), int64(2), object(1)
memory usage: 169.5+ KB


--------

### Duplicated and Unique values:

The following codes shows the unique values quantity in each column of the dataset: 

In [36]:
allowance_backend_table.nunique()

uuid                2882
creation_date       2880
frequency              4
day                   10
updated_at          2882
next_payment_day      31
status                 2
dtype: int64

The total amount of rows coincides with the total *uuid* (unique user id) unique values, there is one row per user id, with a unique *updated_at* date.

The following outputs shows that the *frequency* and *day* columns shares values with the **allowance_events** table's column. The *next_payment_day* have values between 1 and 31, which makes sense since it should contains an integer day value that reference to a specific day in a month.

In [37]:
allowance_backend_table.frequency.unique()

['monthly', 'biweekly', 'weekly', 'daily']
Categories (4, object): ['biweekly', 'daily', 'monthly', 'weekly']

In [38]:
allowance_backend_table.day.unique()

['fifteenth_day', 'first_day', 'friday', 'monday', 'tuesday', 'thursday', 'saturday', 'sunday', 'daily', 'wednesday']
Categories (10, object): ['daily', 'fifteenth_day', 'first_day', 'friday', ..., 'sunday', 'thursday', 'tuesday', 'wednesday']

In [39]:
a=list(allowance_backend_table.next_payment_day.unique())
a.sort()
print(a)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In the following output we can see there are 2 posible status per allowance record, **enabled** and **disable**.

In [40]:
allowance_backend_table.status.unique()

['enabled', 'disabled']
Categories (2, object): ['disabled', 'enabled']

The allowances get created with the status column enabled, but the users can turn them to disabled. In this table the user who **disable** the allowance should not have any active record, as the challenge instructions says. So in the next code we can see all the disabled allowances, and in the following code line those will be left out from the dataset.

There are a total of 763 rows with *disable* status.

In [41]:
allowance_backend_table[allowance_backend_table['status']=='disabled']

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
4,9f5768db-e953-4057-9fa5-264c46293fbd,1726267511,biweekly,friday,2024-10-02T02:33:10.97923183Z,11,disabled
11,9a38d5a4-079d-4300-a34f-d8dd3077532e,1732475627,biweekly,friday,2024-11-24T19:14:11.696320921Z,6,disabled
17,576f559a-3917-4a7c-90cf-94c0d4b8c824,1732812254,monthly,fifteenth_day,2024-11-28T16:54:41.849364978Z,15,disabled
23,8f1c09f4-9ddd-4a67-9215-4c590fb96277,1730046146,monthly,first_day,2024-12-01T15:01:34.155304907Z,1,disabled
28,9e6f29ec-36af-4e32-bffc-505d56bac2de,1723561764,monthly,fifteenth_day,2024-10-05T01:52:47.417673462Z,15,disabled
...,...,...,...,...,...,...,...
2858,fc1dbbbd-09a2-4260-8ea7-11a5cd84711e,1725839521,biweekly,monday,2024-09-08T23:58:25.749091157Z,16,disabled
2871,c5f85d34-50f3-4d78-9259-abb7a9dc73cd,1730232745,weekly,friday,2024-11-28T01:12:00.574956849Z,29,disabled
2874,38ca8952-bd2d-4e33-806f-3b77f0fb02fe,1726591433,biweekly,monday,2024-10-08T22:38:14.787882594Z,14,disabled
2878,adec35e8-3699-45e5-b675-4a6504b296de,1722370541,weekly,friday,2024-08-16T00:48:04.406999984Z,16,disabled


In [42]:
allowance_backend_table=allowance_backend_table[allowance_backend_table['status']=='enabled']

In [43]:
#This code is use to just reset the index, so it starts from 0 again.
allowance_backend_table.reset_index(inplace=True, drop=True)

Preview of the actual dataset:

In [44]:
allowance_backend_table

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
0,30f4e25e-3e37-462e-8c3c-42f24f54350f,1724838709,monthly,fifteenth_day,2024-10-15T08:00:41.445627563Z,15,enabled
1,6da398ad-079d-49b9-8668-6d7ce4d22683,1724659829,monthly,fifteenth_day,1724659829,15,enabled
2,2d30fe2d-6c32-4b8a-a19b-906184f64f62,1731309159,monthly,fifteenth_day,1731309159,15,enabled
3,c30180f3-b35c-470c-b25a-f12ec4ce3340,1725414982,monthly,first_day,2024-10-01T08:00:37.516552581Z,1,enabled
4,35d3d7c2-2a05-4eae-bdf2-7896e611bbc6,1725453772,biweekly,monday,1725453772,16,enabled
...,...,...,...,...,...,...,...
2114,b3b99410-40c0-4574-b6cc-b0769c992971,1726382886,biweekly,wednesday,2024-10-25T08:00:46.017618264Z,6,enabled
2115,c6dde30f-f54d-400c-ad5c-d87728d81731,1730686297,biweekly,saturday,2024-11-23T20:33:08.809249933Z,7,enabled
2116,2e26d7e7-b65e-4468-8096-5bed0ef99988,1729098435,biweekly,monday,2024-11-28T08:00:30.184784321Z,9,enabled
2117,e8f85eea-1974-4000-bde0-2ea7ade70f67,1729636767,biweekly,friday,1729636767,1,enabled


The code below shows that the *updated_at* column and the *creatrion_date* column coincide in 649 rows on the datset. This might happend because the last updated the allowancies settings were modificated was when they were created. In order to prove this I needed to change the columns dtype, so below I would chage it again like it was before.

It is important to notice the format the *creation_date* column has, a number compose by 10 digits, an unknow format for me. The next code shows just 10 of the *updated_at* column values, with only those 10 values we can see that the column stores data in difrent formats, this makes it harder to check if the *next_payment_day* is correct. 

In [45]:
allowance_backend_table['updated_at']= allowance_backend_table['updated_at'].astype('str')
allowance_backend_table['creation_date']= allowance_backend_table['creation_date'].astype('str')
allowance_backend_table[allowance_backend_table['updated_at']== allowance_backend_table['creation_date']]

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status
1,6da398ad-079d-49b9-8668-6d7ce4d22683,1724659829,monthly,fifteenth_day,1724659829,15,enabled
2,2d30fe2d-6c32-4b8a-a19b-906184f64f62,1731309159,monthly,fifteenth_day,1731309159,15,enabled
4,35d3d7c2-2a05-4eae-bdf2-7896e611bbc6,1725453772,biweekly,monday,1725453772,16,enabled
5,9295f1b0-c1cf-4dca-b720-84146395e058,1725501541,biweekly,monday,1725501541,16,enabled
6,c398bb9d-8ba5-4f31-9dd2-97d09a289360,1725453807,biweekly,monday,1725453807,16,enabled
...,...,...,...,...,...,...,...
2102,a515e6dd-797d-4d61-bf90-a01201c7d9a1,1730867822,monthly,first_day,1730867822,1,enabled
2108,5e918355-cb05-4d5c-9b7e-304749df8386,1727635959,weekly,friday,1727635959,4,enabled
2109,b182f209-6433-47b3-bb0b-e5934b5005a7,1727635968,weekly,friday,1727635968,4,enabled
2111,7e3bb803-f698-41b6-a126-b657f138c2dc,1726980961,monthly,first_day,1726980961,1,enabled


In [46]:
allowance_backend_table['updated_at'].head(10)

0    2024-10-15T08:00:41.445627563Z
1                        1724659829
2                        1731309159
3    2024-10-01T08:00:37.516552581Z
4                        1725453772
5                        1725501541
6                        1725453807
7    2024-11-26T08:00:26.493628458Z
8                        1731895648
9    2024-11-28T08:00:26.049462722Z
Name: updated_at, dtype: object

---------------------------

## <CENTER> Join: allowance_events & allowance_backend_table

<CENTER> To compare and correct 

In the next section, I will merge the **allowance_events** and **allowance_backend_table** tables to compare the *TIMESTAMP* and *UPDATED_AT* columns and examine the discrepancies in the *next_paide_day* column.

Firstly, the *user_ID* and the *uuid* must share the same data type, in order to be coincide and be used as a bridge between both tables. Secondly it is the merge, since I want to compare information and check the inconcordance in the **allowance_backend_table**, I'm going to apply an INNER type of join, in order to stay with only those user_id values that can be found in both tables.

In [47]:
# I need both column to have the same dtype, so I can use to join the tables.
allowance_events['user_ID']= allowance_events['user_ID'].astype('string')               
allowance_backend_table['uuid']= allowance_backend_table['uuid'].astype('string')
allowance_merge=allowance_events.merge(allowance_backend_table, how='inner', left_on='user_ID', right_on='uuid', suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)

The followiong information shows that there were only 2114 users id in commun between the tables:

In [48]:
allowance_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2114 entries, 0 to 2113
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_ID             2114 non-null   string        
 1   TIMESTAMP           2114 non-null   datetime64[ns]
 2   allowance_TYPE      2114 non-null   category      
 3   FREQUENCY           2114 non-null   category      
 4   DAY                 2114 non-null   category      
 5   AMOUNT              2114 non-null   int64         
 6   next_payment_day_x  2114 non-null   int64         
 7   uuid                2114 non-null   string        
 8   creation_date       2114 non-null   object        
 9   frequency           2114 non-null   category      
 10  day                 2114 non-null   category      
 11  updated_at          2114 non-null   object        
 12  next_payment_day_y  2114 non-null   int64         
 13  status              2114 non-null   category    

### Date time Columns:

The allowance_backend_table *freceuncy column* column should coincide the *FREQUENCY* column from allowance_events, and it almost did. Sorprisely there is one anomaly with the user_id: **308b2b9c-d49d-4b2d-947c-5b2370da090f**, not only it has two different frequencies values but also diferent *day*, and the *updated_at* and *TIMESTAMP* columns aren't alined as well. Since we considere the **allowance_events** values as truthful, there migth have been an error on the data entery in the **allowance_backend_table** in this peculiar row.

user_id: **308b2b9c-d49d-4b2d-947c-5b2370da090f**

In [49]:
allowance_merge[allowance_merge['frequency']!=allowance_merge['FREQUENCY']]

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
1788,308b2b9c-d49d-4b2d-947c-5b2370da090f,2024-09-09 14:31:44,created,weekly,sunday,2,8,308b2b9c-d49d-4b2d-947c-5b2370da090f,1725917503,monthly,first_day,2024-11-22T02:28:53.912397184Z,1,enabled


In [50]:
allowance_merge[allowance_merge['frequency']==allowance_merge['FREQUENCY']]

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
0,a4fffebe-5b35-4ded-b83b-14852492ddca,2024-12-03 05:07:41,edited,weekly,monday,5,9,a4fffebe-5b35-4ded-b83b-14852492ddca,1733231156,weekly,monday,2024-12-03T13:07:41.05845828Z,9,enabled
1,31f302d1-ee38-4cdc-8943-845f32e23d13,2024-12-03 04:57:13,edited,weekly,tuesday,7,10,31f302d1-ee38-4cdc-8943-845f32e23d13,1726571509,weekly,tuesday,2024-12-03T12:57:12.950212348Z,10,enabled
2,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,2024-12-03 04:28:10,created,monthly,fifteenth_day,20,15,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,1733228890,monthly,fifteenth_day,1733228890,15,enabled
3,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,2024-12-02 22:39:37,created,weekly,friday,11,6,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,1733207977,weekly,friday,1733207977,6,enabled
4,397f8b21-3af1-4a08-81eb-e085eb1160ca,2024-12-02 22:36:34,created,weekly,friday,11,6,397f8b21-3af1-4a08-81eb-e085eb1160ca,1733207794,weekly,friday,1733207794,6,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,d93f143d-830e-4364-88c1-04cadd36b354,2024-07-28 08:02:34,created,biweekly,friday,20,13,d93f143d-830e-4364-88c1-04cadd36b354,1722178953,biweekly,friday,2024-11-29T08:00:54.973036587Z,13,enabled
2110,431a62c4-30ae-405d-95d2-fb7b8284ac4f,2024-07-28 05:20:30,created,biweekly,thursday,5,12,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722169230,biweekly,thursday,2024-08-08T08:00:31.210682807Z,22,enabled
2111,64dd086c-5c33-4af1-a621-0936f01a59bc,2024-07-27 10:22:10,created,biweekly,friday,20,13,64dd086c-5c33-4af1-a621-0936f01a59bc,1722100930,biweekly,friday,1722100930,9,enabled
2112,fb835932-ac1e-45a7-8706-545805841d0c,2024-07-26 17:30:13,created,biweekly,friday,10,13,fb835932-ac1e-45a7-8706-545805841d0c,1722040213,biweekly,friday,1722040213,9,enabled


The exact same thing happens to the columns *day* and *DAY*:

In [51]:
allowance_merge[allowance_merge['day']==allowance_merge['DAY']]

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
0,a4fffebe-5b35-4ded-b83b-14852492ddca,2024-12-03 05:07:41,edited,weekly,monday,5,9,a4fffebe-5b35-4ded-b83b-14852492ddca,1733231156,weekly,monday,2024-12-03T13:07:41.05845828Z,9,enabled
1,31f302d1-ee38-4cdc-8943-845f32e23d13,2024-12-03 04:57:13,edited,weekly,tuesday,7,10,31f302d1-ee38-4cdc-8943-845f32e23d13,1726571509,weekly,tuesday,2024-12-03T12:57:12.950212348Z,10,enabled
2,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,2024-12-03 04:28:10,created,monthly,fifteenth_day,20,15,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,1733228890,monthly,fifteenth_day,1733228890,15,enabled
3,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,2024-12-02 22:39:37,created,weekly,friday,11,6,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,1733207977,weekly,friday,1733207977,6,enabled
4,397f8b21-3af1-4a08-81eb-e085eb1160ca,2024-12-02 22:36:34,created,weekly,friday,11,6,397f8b21-3af1-4a08-81eb-e085eb1160ca,1733207794,weekly,friday,1733207794,6,enabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,d93f143d-830e-4364-88c1-04cadd36b354,2024-07-28 08:02:34,created,biweekly,friday,20,13,d93f143d-830e-4364-88c1-04cadd36b354,1722178953,biweekly,friday,2024-11-29T08:00:54.973036587Z,13,enabled
2110,431a62c4-30ae-405d-95d2-fb7b8284ac4f,2024-07-28 05:20:30,created,biweekly,thursday,5,12,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722169230,biweekly,thursday,2024-08-08T08:00:31.210682807Z,22,enabled
2111,64dd086c-5c33-4af1-a621-0936f01a59bc,2024-07-27 10:22:10,created,biweekly,friday,20,13,64dd086c-5c33-4af1-a621-0936f01a59bc,1722100930,biweekly,friday,1722100930,9,enabled
2112,fb835932-ac1e-45a7-8706-545805841d0c,2024-07-26 17:30:13,created,biweekly,friday,10,13,fb835932-ac1e-45a7-8706-545805841d0c,1722040213,biweekly,friday,1722040213,9,enabled


Now, lets observe the *TIEMSTAMP* and *updated_at* similarities and differences:

As we can see in this dataset sample, when the *updated_at* value is saved as a Date Time and Zone value, some maches with the data stored in the *TIMESTAMP* column. This will be useful in case we wanted to normalize the *updated_at* column, and in case we needed it to calculate or verify the *next_payment_day*. This discordancies between the columns proves that the *updated_at* column code to save the data is not being normalize, it doesn't have a determinate format to store the data, this might cause more more discordancies if their values are being used to calclulate other column like *next_payment_day*. 

In [52]:
allowance_merge[['TIMESTAMP','updated_at']]

,TIMESTAMP,updated_at
0,2024-12-03 05:07:41,2024-12-03T13:07:41.05845828Z
1,2024-12-03 04:57:13,2024-12-03T12:57:12.950212348Z
2,2024-12-03 04:28:10,1733228890
3,2024-12-02 22:39:37,1733207977
4,2024-12-02 22:36:34,1733207794
...,...,...
2109,2024-07-28 08:02:34,2024-11-29T08:00:54.973036587Z
2110,2024-07-28 05:20:30,2024-08-08T08:00:31.210682807Z
2111,2024-07-27 10:22:10,1722100930
2112,2024-07-26 17:30:13,1722040213


### Column : *next_peyment_day*
Now, we are going to analize the discrepancies and inconsistencies in the *next_paymente_day_y* (belognws to the allowance_backend_table):

1. From 2114 rows, 993 have incorrected next_paymet_day.

In [53]:
allowance_merge[allowance_merge['next_payment_day_x']!=allowance_merge['next_payment_day_y']].shape

(993, 14)

In [54]:
allowance_merge.head(6)

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
0,a4fffebe-5b35-4ded-b83b-14852492ddca,2024-12-03 05:07:41,edited,weekly,monday,5,9,a4fffebe-5b35-4ded-b83b-14852492ddca,1733231156,weekly,monday,2024-12-03T13:07:41.05845828Z,9,enabled
1,31f302d1-ee38-4cdc-8943-845f32e23d13,2024-12-03 04:57:13,edited,weekly,tuesday,7,10,31f302d1-ee38-4cdc-8943-845f32e23d13,1726571509,weekly,tuesday,2024-12-03T12:57:12.950212348Z,10,enabled
2,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,2024-12-03 04:28:10,created,monthly,fifteenth_day,20,15,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,1733228890,monthly,fifteenth_day,1733228890,15,enabled
3,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,2024-12-02 22:39:37,created,weekly,friday,11,6,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,1733207977,weekly,friday,1733207977,6,enabled
4,397f8b21-3af1-4a08-81eb-e085eb1160ca,2024-12-02 22:36:34,created,weekly,friday,11,6,397f8b21-3af1-4a08-81eb-e085eb1160ca,1733207794,weekly,friday,1733207794,6,enabled
5,2a18265a-128b-446e-aba2-045e38bd66bc,2024-12-02 21:14:13,created,monthly,fifteenth_day,5,15,2a18265a-128b-446e-aba2-045e38bd66bc,1733202853,monthly,fifteenth_day,1733202853,15,enabled


Lets see some row:


There is no Monday 5th this month (dicember 2024), so there is no doudt that here the *nest_paymente_day_y* value is incorrect.


In [55]:
allowance_merge[allowance_merge.index==2113]

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
2113,17df1db7-0a6a-4661-90d6-949b601ab02b,2024-07-26 13:34:37,created,biweekly,monday,20,16,17df1db7-0a6a-4661-90d6-949b601ab02b,1722026076,biweekly,monday,1722026076,5,enabled


Here there is a real date, Monday the 2nd, but as we declare before, today is Tuesday, December 3rd. The column is supposed to save the **next** payment day, and in this case there is a date that has already passed, so it's also incorrect.

In [56]:
allowance_merge[allowance_merge.index==1981]

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
1981,744ccd67-dcb5-4276-965f-c924e43042b7,2024-08-23 22:39:56,created,biweekly,monday,20,16,744ccd67-dcb5-4276-965f-c924e43042b7,1724477996,biweekly,monday,1724477996,2,enabled


One hypotesis:

Seing this examples I can think about two rasons why the result is incorrect:
1. Sometimes while calculating this type of values like next_payment_day, there is used some programming method that asigned a numer to a day of the week, like for example:
- 1 becomes Sunday
- 2 becomes Monday
- 3 becomes Tuesday
- 4 becomes Wednesday
- 5 becomes Thursday
- 6 becomes Friday
- 7 becomes Saturday
Perhaps when calculating the next day of the week, the value of the *day* column is taken into account to find the number that represents it. For example, in record 2113, the value of *day* is 'monday', so it is assigned the number 2, so when calculating the next date, the current date (December 3rd) is added 2, resulting in 5. What this formula does not take into account is that the current day is Tuesday, so you would have to add 6 to actually calculate next Monday.

2. The second rasons is related to the fact that the *updated_at* column (in which the *next_payment_day_y* could be base on to calculate its value), has different formats, so the code might not work in the same way for each format.

In [57]:
allowance_merge[allowance_merge.index==2113]

,user_ID,TIMESTAMP,allowance_TYPE,FREQUENCY,DAY,AMOUNT,next_payment_day_x,uuid,creation_date,frequency,day,updated_at,next_payment_day_y,status
2113,17df1db7-0a6a-4661-90d6-949b601ab02b,2024-07-26 13:34:37,created,biweekly,monday,20,16,17df1db7-0a6a-4661-90d6-949b601ab02b,1722026076,biweekly,monday,1722026076,5,enabled


In [58]:
allowance_merge.columns

Index(['user_ID', 'TIMESTAMP', 'allowance_TYPE', 'FREQUENCY', 'DAY', 'AMOUNT',
       'next_payment_day_x', 'uuid', 'creation_date', 'frequency', 'day',
       'updated_at', 'next_payment_day_y', 'status'],
      dtype='object')

In [59]:
allowance_backend_table_clean=allowance_merge[['uuid','creation_date','FREQUENCY', 'DAY','TIMESTAMP','next_payment_day_x','status']]

In [60]:
allowance_backend_table_clean.rename(columns={'FREQUENCY': 'frequency', 'DAY': 'day', 'TIMESTAMP': 'updated_at', 'next_payment_day_x':'next_payment_day'}, inplace=True)

C:\Users\trini\AppData\Local\Temp\ipykernel_14580\1725933774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allowance_backend_table_clean.rename(columns={'FREQUENCY': 'frequency', 'DAY': 'day', 'TIMESTAMP': 'updated_at', 'next_payment_day_x':'next_payment_day'}, inplace=True)


In case the bakend team needs it, the following code will export the data recolected and cleaned and with only enabled rows, for the allowance_backend_table:

In [61]:
allowance_backend_table_clean.to_csv('allowance_backend_table_clean.CSV', sep=',',index=False)

-------------

## <CENTER> Join: allowance_backend_table & payment_schedule_backend_table

<CENTER> To compare and correct

In [62]:
payment_schedule_backend_table['user_id']= payment_schedule_backend_table['user_id'].astype('string')
allowance_payment_merge=allowance_backend_table_clean.merge(payment_schedule_backend_table, how='inner', left_on='uuid', right_on='user_id', suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)

In [63]:
allowance_payment_merge

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status,user_id,payment_date
0,a4fffebe-5b35-4ded-b83b-14852492ddca,1733231156,weekly,monday,2024-12-03 05:07:41,9,enabled,a4fffebe-5b35-4ded-b83b-14852492ddca,9
1,31f302d1-ee38-4cdc-8943-845f32e23d13,1726571509,weekly,tuesday,2024-12-03 04:57:13,10,enabled,31f302d1-ee38-4cdc-8943-845f32e23d13,10
2,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,1733228890,monthly,fifteenth_day,2024-12-03 04:28:10,15,enabled,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,15
3,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,1733207977,weekly,friday,2024-12-02 22:39:37,6,enabled,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,6
4,397f8b21-3af1-4a08-81eb-e085eb1160ca,1733207794,weekly,friday,2024-12-02 22:36:34,6,enabled,397f8b21-3af1-4a08-81eb-e085eb1160ca,6
...,...,...,...,...,...,...,...,...,...
2109,d93f143d-830e-4364-88c1-04cadd36b354,1722178953,biweekly,friday,2024-07-28 08:02:34,13,enabled,d93f143d-830e-4364-88c1-04cadd36b354,13
2110,431a62c4-30ae-405d-95d2-fb7b8284ac4f,1722169230,biweekly,thursday,2024-07-28 05:20:30,12,enabled,431a62c4-30ae-405d-95d2-fb7b8284ac4f,22
2111,64dd086c-5c33-4af1-a621-0936f01a59bc,1722100930,biweekly,friday,2024-07-27 10:22:10,13,enabled,64dd086c-5c33-4af1-a621-0936f01a59bc,9
2112,fb835932-ac1e-45a7-8706-545805841d0c,1722040213,biweekly,friday,2024-07-26 17:30:13,13,enabled,fb835932-ac1e-45a7-8706-545805841d0c,9


Since we have already checked and corrected the *next_payment_day* column, lets compare it with *paymnet_day:*

On the next output code we can observe that there are 1114 rows that shares their *next_payment_day* and *payment_date* values, which means there are exactly 1000 incorrect paymente_date. 

In [64]:
allowance_payment_merge[allowance_payment_merge['next_payment_day']==allowance_payment_merge['payment_date']]

,uuid,creation_date,frequency,day,updated_at,next_payment_day,status,user_id,payment_date
0,a4fffebe-5b35-4ded-b83b-14852492ddca,1733231156,weekly,monday,2024-12-03 05:07:41,9,enabled,a4fffebe-5b35-4ded-b83b-14852492ddca,9
1,31f302d1-ee38-4cdc-8943-845f32e23d13,1726571509,weekly,tuesday,2024-12-03 04:57:13,10,enabled,31f302d1-ee38-4cdc-8943-845f32e23d13,10
2,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,1733228890,monthly,fifteenth_day,2024-12-03 04:28:10,15,enabled,60d0d14e-33e6-45ae-b7f3-c5c1d732f734,15
3,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,1733207977,weekly,friday,2024-12-02 22:39:37,6,enabled,1fc74dc2-7d96-4a2a-a8c0-ae3a61eacf7d,6
4,397f8b21-3af1-4a08-81eb-e085eb1160ca,1733207794,weekly,friday,2024-12-02 22:36:34,6,enabled,397f8b21-3af1-4a08-81eb-e085eb1160ca,6
...,...,...,...,...,...,...,...,...,...
2097,1cde5073-f2b6-4f02-8351-5ebed64e6dec,1722452110,weekly,friday,2024-07-31 11:55:10,6,enabled,1cde5073-f2b6-4f02-8351-5ebed64e6dec,6
2100,155e5500-79ad-4a9d-ab4d-08295e7c491a,1722387427,biweekly,friday,2024-07-30 17:57:07,13,enabled,155e5500-79ad-4a9d-ab4d-08295e7c491a,13
2104,c440479e-8cf2-402e-8da9-9c682ea3df39,1721931946,biweekly,thursday,2024-07-29 14:38:26,12,enabled,c440479e-8cf2-402e-8da9-9c682ea3df39,12
2108,0dc27888-e0fd-4785-999a-e2b5b8bbe686,1722184173,monthly,fifteenth_day,2024-07-28 09:29:34,15,enabled,0dc27888-e0fd-4785-999a-e2b5b8bbe686,15


Since we have already clean and calculated the correct payment day, I can extract those values and insert them into the payment_schedule_backend_table. In case the bakend team needs it, the following code will export the data recolected and cleaned and with only the user_id and the payment_day for the payment_schedule_backend_table.

In [65]:
payment_schedule_backend_table_clean=allowance_payment_merge[['user_id','next_payment_day']]
payment_schedule_backend_table_clean.rename(columns={'next_payment_day': 'payment_date'}, inplace=True)


C:\Users\trini\AppData\Local\Temp\ipykernel_14580\508757132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_schedule_backend_table_clean.rename(columns={'next_payment_day': 'payment_date'}, inplace=True)


In [66]:
payment_schedule_backend_table_clean.to_csv('payment_schedule_backend_table_clean.CSV', sep=',',index=False)

## Conclusion

After having extracted and explored the information from each dataset, and having been able to relate the tables to each other, I conclude by reporting that the biggest problem is found in the columns: *updated_at* and *next_payment_day*.

The payment_schedule_backend_table table consists of 13 repeated records, this can cause an issue since each record has a different payday date. These are the IDs:

d299d804-0d05-48c8-ad62-601d54438e34, 72e8fe3f-0dd9-44e3-8775-660bacb17310, 19d933a6-2810-4c08-b3d6-d68ef3c6eac3,c091a91b-c6fd-44cb-8ac7-fe3fdbafccea, 72190913-1208-4333-aaf4-ea7d04dccacd, 9221dccf-44f3-446d-b3ce-19f9123ea627, 172fe923-347e-4d0e-ba82-1e9969aaadde, abf5cf1e-e81a-4b5f-9d5b-649fa10d8f81, 42f61042-3d31-42ff-9f1a-508716069464, d769685e-9046-4ffe-a51d-829ec7f1d877, eab4d50f-2a17-4f07-b462-53be0e54f6c6, 77cdda96-6e5b-47c1-b486-9f06da2c7372, 72e8fe3f-0dd9-44e3-8775-660bacb17310	

The code that stores the information in *next_payment_day* of the table **allowance_backend_table** and *payment_day* of **payment_schedule_backend_table**, should be thoroughly reviewed, judging by the fact that not all of their values ​​are wrong, I consider that it may be a calculation error where the day of the week of the current day is probably not taken into account.


Within the repository you can also find two csv files with the ending '_clean', where I decided to save the clean backend tables, in case they are of help (it should be noted that the allowance backend_table_clean table only keeps records with status enabled).
